In [ ]:
#!pip install transformers


## Sentence Transformer to get Embeddings and Metadata for EmbeddingProjector
[EmbeddingProjector](https://projector.tensorflow.org/)

In [ ]:
!pip install -U sentence-transformers

In [7]:
from sentence_transformers import SentenceTransformer
model_st = SentenceTransformer('all-distilroberta-v1')
cat /usr/local/cuda/version.txt

In [43]:
df = pd.read_csv("datasets/problem_statements.csv")
df.head()

,Unnamed: 0,PMID,source,Title,text,labels,DOI
0,0,NaN,acl_cambridge,NaN,The difficulty with this task lies in the fact...,1,NaN
1,1,NaN,acl_cambridge,NaN,The problem with rich annotations is that they...,1,NaN
2,2,NaN,acl_cambridge,NaN,"As a consequence , when adapting existing meth...",1,NaN
3,3,NaN,acl_cambridge,NaN,The second problem of traditional word alignme...,1,NaN
4,4,NaN,acl_cambridge,NaN,The main drawback of these systems is that the...,1,NaN


In [44]:
vectors = [model_st.encode(sentence) for sentence in df["text"].values]


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [45]:
len(vectors)

3138

Saving files to download

In [46]:
pd.DataFrame(vectors).to_csv("temp/vectors.csv", index=False, header=False, sep="\t")

In [47]:
df.to_csv("temp/metadata.csv", index=False, sep="\t")

In [48]:
from IPython.display import HTML
import base64
def create_download_link( df, title = "Download CSV file", filename = "data.csv"):  
    csv = df
    if type(df) != str:
        csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

## For Longform Passages

In [ ]:
!pip install transformers
!pip install torch==1.2.0 torchvision==0.4.0 #had to find this specification in the archives ... CUDA

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel

df = pd.read_csv("datasets/problem_statements.csv")

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096") #longformer because it's a paragraph ... use sentence transformers for shorter texts
model = AutoModel.from_pretrained("allenai/longformer-base-4096")

#embeddings = df["text"].map(lambda t: model(**tokenizer(t, return_tensors="pt")))
# text = "Spaces is an S3-compatible object storage service that lets you store and serve large amounts of data. Each Space is a bucket for you to store and serve files. The free, built-in Spaces CDN minimizes page load times, improves performance, and reduces bandwidth and infrastructure costs."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)
#encoded_input, output

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
metadata = df#[["name", "title", "url", "visit_time"]]
metadata.to_csv("metadata.tsv", index=False, sep="\t")

vectors = [model(**tokenizer(document[:4096], return_tensors='pt'))[1].squeeze(0).detach() for document in df["text"].values]

vectors_np = [v.numpy() for v in vectors]
len(vectors_np)
vectors_np[0]
pd.DataFrame(vectors_np).to_csv("vectors.tsv", index=False, header=False, sep="\t")